In [1]:
import os
import time
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import torch

import viz
import utils
from preprocess import get_preprocess
from explainer.sparse import SparseExplainer

image_path = 'images/elephant.png'
image_class = 101  # tusker
raw_img = viz.pil_loader(image_path)
plt.figure(figsize=(5, 5))
plt.imshow(raw_img)
plt.axis('off')
plt.title('class_id=101 (Tusker)')

model_name = 'resnet50'
method_name = 'sparse'
show_style = 'imshow'
transf = get_preprocess(model_name, method_name)
model = utils.load_model(model_name)
model.cuda()

/fs/clip-ml/shifeng/anaconda3/envs/torch-0.3.1/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
  

In [3]:
explainer = SparseExplainer(model)
        
inp = transf(raw_img)
inp = utils.cuda_var(inp.unsqueeze(0), requires_grad=True)
saliency = explainer.explain(inp)
saliency = utils.upsample(saliency, (raw_img.height, raw_img.width))
saliency_map = saliency.cpu().numpy()

[0.] [0.] [0.] [0.]
[0.00067835] [3.1465498e-07] [9.9999525e-05] [9.999903e-09]
[0.00954802] [5.609127e-05] [9.4713425e-05] [9.052822e-09]
[0.01749875] [0.00018808] [5.5147695e-05] [3.3130692e-09]
[0.02292635] [0.00032192] [1.3673572e-05] [5.8319755e-10]
[0.01710916] [0.00018884] [4.184804e-05] [2.25649e-09]
[0.01159943] [9.5080104e-05] [4.9414783e-05] [3.6012966e-09]
[0.00759728] [4.8060574e-05] [4.130363e-05] [3.4166912e-09]
[0.00504904] [2.7553211e-05] [3.6994366e-05] [2.5744398e-09]
[0.00603464] [3.325528e-05] [2.9007666e-05] [1.628323e-09]


In [13]:
show_style = 'imshow'
plt.figure(figsize=(25, 15))
plt.imshow(raw_img)
plt.axis('off')
plt.title('class_id=101 (Tusker)')
if show_style == 'camshow':
    viz.plot_cam(np.abs(saliency).max(axis=1).squeeze(),
                 raw_img, 'jet', alpha=0.5)
else:
    saliency = saliency.squeeze().transpose(0, 2)
    saliency -= saliency.min()
    saliency /= (saliency.max() + 1e-20)
    plt.imshow(saliency, cmap='gray')

plt.axis('off')
# plt.tight_layout()
# plt.savefig('images/tusker_saliency.png')

AttributeError: 'torch.cuda.FloatTensor' object has no attribute 'swapaxes'